In [4]:
import tensorflow as tf

TensorFlow operates on multidimensional arrays or tensors, $T_{\mu_1 \mu_2 \mu_3 ... \mu_n}$ represented as tf.Tensor objects. For instance, a rank 1 tensor, $x_\mu$, can be created from an array:

In [20]:
x = tf.constant([1, 2])
print(x.shape)
print(x.dtype)
print(tf.rank(x))

(2,)
<dtype: 'int32'>
tf.Tensor(1, shape=(), dtype=int32)


and operated on an it as follows:

In [21]:
y = x + 1
print(y)

tf.Tensor([2 3], shape=(2,), dtype=int32)


A rank 2 tensor, $A_{\mu \nu}$, can be created from a matrix

In [26]:
A = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
print(A.shape)
print(tf.rank(A))

(2, 2)
tf.Tensor(2, shape=(), dtype=int32)


and operaerated on as follows:

In [23]:
A @ A # TensorFlow's dot product operator

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 7., 10.],
       [15., 22.]], dtype=float32)>

In [ ]:
B = tf.constant([[5, 6], [7, 8]], dtype = tf.float32)
# Tensorflow's matrix multiplication operator
print(A @ B) 
print(B @ A)

tf.Tensor(
[[19. 22.]
 [43. 50.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[23. 34.]
 [31. 46.]], shape=(2, 2), dtype=float32)


In [38]:
A_inv = tf.linalg.inv(A)
print(tf.round(A @ A_inv))

tf.Tensor(
[[ 1.  0.]
 [-0.  1.]], shape=(2, 2), dtype=float32)


In [ ]:
sigma = tf.constant([[1.0, -1.0]], dtype=tf.float32)
print(sigma)
print(tf.transpose(sigma))
print(sigma @ tf.transpose(sigma))
print(tf.transpose(sigma) @ sigma)

tf.Tensor([[ 1. -1.]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[ 1.]
 [-1.]], shape=(2, 1), dtype=float32)
tf.Tensor([[2.]], shape=(1, 1), dtype=float32)
tf.Tensor(
[[ 1. -1.]
 [-1.  1.]], shape=(2, 2), dtype=float32)


TensorFlow is great at running things in parallel using your GPU.

In [45]:
if tf.config.list_physical_devices('GPU'):
  print("TensorFlow **IS** using the GPU")
else:
  print("TensorFlow **IS NOT** using the GPU")

TensorFlow **IS NOT** using the GPU


In [46]:
tf.config.list_physical_devices('GPU')

[]